In [ ]:
!pip install tensorflow==2.9.0
!pip install tensorflow-text==2.9.0

  Using cached tensorflow_text-2.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
  Using cached tf_keras-2.16.0-py3-none-any.whl (1.7 MB)
  Using cached tf_keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.15.1
    Uninstalling tf_keras-2.15.1:
      Successfully uninstalled tf_keras-2.15.1


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle
import joblib
import time

# 1. 데이터 로드

In [ ]:
# 왓챠피디아 크롤링 데이터
data = pd.read_csv('/content/media_data.csv', encoding = 'utf-8')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18405 entries, 0 to 18404
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           18405 non-null  object 
 1   Runtime         18405 non-null  float64
 2   Release Date    18405 non-null  object 
 3   Certification   15783 non-null  object 
 4   Genres          18405 non-null  object 
 5   Origin Country  18334 non-null  object 
 6   Overview        18405 non-null  object 
 7   Director        15274 non-null  object 
 8   Cast            17855 non-null  object 
 9   Providers       16331 non-null  object 
 10  Rating Value    18405 non-null  float64
 11  Rating Count    18405 non-null  int64  
 12  Poster URL      18405 non-null  object 
 13  Backdrop URLs   18405 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 2.0+ MB


In [ ]:
# 필요한 컬럼(피처)만 선택해서 새로운 데이터셋 생성
content = data[['Title', 'Genres', 'Overview', 'Rating Value', 'Rating Count']]

print(content.head())
print(content.columns)

        Title             Genres  \
0     성냥공장 소녀           코미디, 드라마   
1     키다리 아저씨    코미디, 애니메이션, 드라마   
2  주윤발의 행운의 별  코미디, 로맨스, 로맨틱 코미디   
3        부귀병단            코미디, 액션   
4        불가사리    SF, 액션, 공포, 코미디   

                                            Overview  Rating Value  \
0  무능력하고 무표정한 얼굴의 엄마와 계부의 생활비를 위해 매일같이 성냥공장에서 기계처...           3.8   
1  고아이지만 언제나 밝은 주디와 그런 주디를 도와주는 후견인 '키다리 아저씨'의 사랑...           4.2   
2  재벌가 도련님 임보생은 재산 상속권을 조건으로 육촌 동생 진옥선과의 혼인을 강요당한...           3.2   
3  1940년대 초, 일본의 침공으로 전 중국이 혼란에 빠져있을 무렵, 중국내 일본의 ...           2.9   
4  네바다주 사막 한 가운데 있는 작은 마을에는 20명도 안되는 주민들이 서로 도우며 ...           3.3   

   Rating Count  
0          5278  
1          1623  
2           184  
3           128  
4         61592  
Index(['Title', 'Genres', 'Overview', 'Rating Value', 'Rating Count'], dtype='object')


In [ ]:
content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18405 entries, 0 to 18404
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         18405 non-null  object 
 1   Genres        18405 non-null  object 
 2   Overview      18405 non-null  object 
 3   Rating Value  18405 non-null  float64
 4   Rating Count  18405 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 719.1+ KB


In [ ]:
# MBTI 500 데이터
mbti = pd.read_csv('/content/MBTI 500.csv')
print(mbti.info())

# 2. LaBSE 모델 로드 및 임베딩

- 사전 학습된 LaBSE 모델을 사용하여 텍스트 데이터를 고차원 벡터로 변환합니다.
- 이 임베딩 모델은 다국어 문장 임베딩을 위한 모델로, 15개 언어만 사용하도록 하여 경량화된 모델입니다.
- 고차원 벡터로 변환된 텍스트는 유사도 계산에 사용됩니다.

In [ ]:
# LaBSE 모델 로드
encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang/1")
preprocessor = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang_preprocess/1")

# 텍스트를 고차원 벡터로 인코딩하는 모델 구성
def build_embedding_model():
    sentences = tf.keras.layers.Input(shape=(), dtype=tf.string, name="sentences")
    encoder_inputs = preprocessor(sentences)
    sentence_representation = encoder(encoder_inputs)["pooled_output"]
    normalized_sentence_representation = tf.nn.l2_normalize(sentence_representation, axis=-1)  # for cosine similarity
    return tf.keras.Model(sentences, normalized_sentence_representation)

embedding_model = build_embedding_model()
embedding_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sentences (InputLayer)         [(None,)]            0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_type_ids':   0           ['sentences[0][0]']              
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [ ]:
# 텍스트 임베딩 함수
def embed_texts(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_tensors = tf.constant(batch_texts)
        batch_embeddings = embedding_model(batch_tensors).numpy()
        embeddings.extend(batch_embeddings)
    return embeddings

In [ ]:
# MBTI 임베딩
mbti_posts = mbti.groupby('type')['posts'].apply(lambda posts: ' '.join(posts)).tolist()
mbti_embeddings = embed_texts(mbti_posts)
mbti_index = mbti.groupby('type').groups.keys()
mbti_embeddings_dict = dict(zip(mbti_index, mbti_embeddings))

# 콘텐츠 줄거리 임베딩
contents_texts = content['Overview'].tolist()
contents_embeddings = embed_texts(contents_texts)
contents_index = content['Title'].tolist()
contents_embeddings_dict = dict(zip(contents_index, contents_embeddings))

In [ ]:
# 콘텐츠 임베딩 파일 저장 및 로드
with open('contents_embeddings_dict.pkl', 'wb') as f:
    pickle.dump(contents_embeddings_dict, f)

In [ ]:
# MBTI 임베딩 파일 저장 및 로드
with open('mbti_embeddings_dict.pkl', 'rb') as f:
    mbti_embeddings_dict = pickle.load(f)

In [ ]:
# 콘텐츠 임베딩 로드
with open('contents_embeddings_dict_0612.pkl', 'rb') as f:
    contents_embeddings_dict = pickle.load(f)

# 3. 콘텐츠 장르 원-핫 인코딩 / 데이터 표준화 / 콘텐츠 평점 개수 정규화

- 콘텐츠 장르를 원-핫 인코딩하여 장르 정보를 벡터화합니다.
- 원-핫 인코딩된 장르 벡터와 평점 데이터를 결합하여 학습 데이터를 생성합니다.
- 데이터 표준화를 통해 모델 학습을 용이하게 합니다.
- 대중성 점수로 사용하기 위해 콘텐츠 평점 개수를 정규화합니다.

In [ ]:
# 콘텐츠 장르 원-핫 인코딩
genres = content['Genres'].str.get_dummies(sep=', ')

# 평점 데이터 결합
cbf_model_input = np.hstack([genres.values, content['Rating Value'].values.reshape(-1, 1)])

# 데이터 표준화
cbf_scaler = StandardScaler()
cbf_model_input_scaled = cbf_scaler.fit_transform(cbf_model_input)

In [ ]:
# 콘텐츠 평점 개수를 정규화하는 함수
def normalize_popularity_score(content):
    scaler = MinMaxScaler()
    content['Normalized Popularity Score'] = scaler.fit_transform(content[['Rating Count']])
    return content

content = normalize_popularity_score(content)

<ipython-input-11-e69302caf052>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['Normalized Popularity Score'] = scaler.fit_transform(content[['Rating Count']])


# 4. 모델 학습 및 최적화

- 학습 데이터와 평가 데이터로 분리합니다.
- GBM(Gradient Boosting Machine)을 사용하여 모델을 학습합니다.
- Grid Search를 통해 최적의 하이퍼파라미터를 탐색하고, 최적의 모델을 선정합니다.

In [ ]:
# 학습 데이터와 평가 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(cbf_model_input_scaled, content['Rating Value'].values, test_size=0.2, random_state=42)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': [0.6, 0.8, 1.0]
}

# 그리드 서치 설정
gbm = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=1, verbose=2)

# 그리드 서치 수행
grid_search.fit(X_train, y_train)

# 최적의 모델 출력
best_gbm = grid_search.best_estimator_
print(f'Best GBM Model: {best_gbm}')

# 검증 데이터로 모델 성능 평가
y_pred = best_gbm.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f'Validation MSE: {mse}')

# 모델 저장
joblib.dump(best_gbm, 'best_gbm_model_final.pkl')

In [ ]:
# 최적의 모델 로드
best_gbm = joblib.load('best_gbm_model_final_0612.pkl')

# 5. 콘텐츠 추천 시스템 구현

- **유사도 계산 함수**: 두 임베딩 벡터 간의 코사인 유사도를 계산합니다.
- **MBTI 기반 추천**: 사용자의 MBTI 임베딩과 콘텐츠 임베딩 간의 유사도를 계산하여 콘텐츠를 추천합니다.
- **선호 콘텐츠 기반 추천**: 사용자가 선호하는 콘텐츠와 유사한 콘텐츠를 추천합니다.

In [ ]:
# 콘텐츠와 MBTI 유형 간의 유사도 계산 함수
def calculate_similarity(content_embedding, user_embedding):
    from numpy import dot
    from numpy.linalg import norm
    return dot(content_embedding, user_embedding) / (norm(content_embedding) * norm(user_embedding))

# 1. MBTI 임베딩 기반 추천
def recommend_contents_by_mbti(user_embedding, content_embeddings_dict, top_n=100):
    recommended_contents = []
    for content_id, embedding in content_embeddings_dict.items():
        similarity = calculate_similarity(embedding, user_embedding)
        recommended_contents.append((content_id, similarity))
    recommended_contents = sorted(recommended_contents, key=lambda x: x[1], reverse=True)
    return recommended_contents[:top_n]

# 2. 선호 콘텐츠 유사도 기반 추천
def recommend_similar_contents(preferred_contents, content_embeddings_dict, top_n=100):
    similar_contents = {}
    for content in preferred_contents:
        content_embedding = content_embeddings_dict[content]
        for other_content, embedding in content_embeddings_dict.items():
            if other_content not in preferred_contents:
                similarity = calculate_similarity(embedding, content_embedding)
                if other_content not in similar_contents:
                    similar_contents[other_content] = 0
                similar_contents[other_content] += similarity
    similar_contents = sorted(similar_contents.items(), key=lambda x: x[1], reverse=True)
    return similar_contents[:top_n]

- **결합 추천 시스템**: MBTI 유사도 점수, 선호 콘텐츠 유사도 점수, 모델 점수, 대중성 점수에 각각 가중치를 부여하여 콘텐츠를 추천합니다.

In [ ]:
# 결합 추천 시스템
def recommend_contents_combined(user_embedding, preferred_contents, content_embeddings_dict, model, content, cbf_model_input_scaled, top_n=20):
    weight_mbti = 0.3
    weight_similar = 0.4
    weight_model = 0.1
    weight_popularity = 0.2

    mbti_recommendations = recommend_contents_by_mbti(user_embedding, content_embeddings_dict, top_n=100)
    similar_content_recommendations = recommend_similar_contents(preferred_contents, content_embeddings_dict, top_n=100)

    combined_recommendations = {}

    # MBTI 유사도 점수 보정
    mbti_scores = [score for _, score in mbti_recommendations]
    mbti_std = np.std(mbti_scores)
    mbti_scores_normalized = [score / mbti_std for score in mbti_scores]

    for (content_id, _), score in zip(mbti_recommendations, mbti_scores_normalized):
        if content_id not in combined_recommendations:
            combined_recommendations[content_id] = 0
        combined_recommendations[content_id] += weight_mbti * score

    # 선호 콘텐츠 유사도 점수 보정
    similar_scores = [score for _, score in similar_content_recommendations]
    similar_std = np.std(similar_scores)
    similar_scores_normalized = [score / similar_std for score in similar_scores]

    for (content_id, _), score in zip(similar_content_recommendations, similar_scores_normalized):
        if content_id not in combined_recommendations:
            combined_recommendations[content_id] = 0
        combined_recommendations[content_id] += weight_similar * score

    # 모델 점수 보정
    content_indices = [content.index[content['Title'] == content_id][0] for content_id in combined_recommendations.keys()]
    if content_indices:
        content_features = cbf_model_input_scaled[content_indices]
        model_scores = model.predict(content_features).flatten()
        model_std = np.std(model_scores)
        model_scores_normalized = [score / model_std for score in model_scores]

        for content_id, model_score in zip(combined_recommendations.keys(), model_scores_normalized):
            combined_recommendations[content_id] += weight_model * model_score

    # 대중성 점수 보정
    popularity_scores = [content.loc[content['Title'] == content_id, 'Normalized Popularity Score'].values[0] for content_id in combined_recommendations.keys()]
    popularity_std = np.std(popularity_scores)
    popularity_scores_normalized = [score / popularity_std for score in popularity_scores]

    for content_id, popularity_score in zip(combined_recommendations.keys(), popularity_scores_normalized):
        combined_recommendations[content_id] += weight_popularity * popularity_score

    filtered_recommendations = {content_id: score for content_id, score in combined_recommendations.items()
                                if content.loc[content['Title'] == content_id, 'Rating Value'].values[0] >= 3.8}

    final_recommendations = sorted(filtered_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [content_id for content_id, _ in final_recommendations[:top_n]]

- 재추천 시스템: 첫 번째 추천 목록에서 사용자가 마음에 드는(선택한) 콘텐츠를 기반으로 이전에 추천된 콘텐츠와 처음에 선호 콘텐츠로 선택한 콘텐츠를 제외하고 재추천을 수행합니다.

In [ ]:
# 재추천 시스템
def recommend_contents_combined_excluding(user_embedding, preferred_contents, previous_recommendations, initial_preferred_contents, content_embeddings_dict, model, content, cbf_model_input_scaled, top_n=20):
    # 가중치 설정
    weight_mbti = 0.3
    weight_similar = 0.4
    weight_model = 0.1
    weight_popularity = 0.2

    # 초기 선호 콘텐츠와 현재 선호 콘텐츠 합치기
    all_preferred_contents = list(set(preferred_contents + initial_preferred_contents))

    # MBTI와 유사 콘텐츠 추천 상위 100개 가져오기
    mbti_recommendations = recommend_contents_by_mbti(user_embedding, content_embeddings_dict, top_n=100)
    similar_content_recommendations = recommend_similar_contents(all_preferred_contents, content_embeddings_dict, top_n=100)

    # 추천 콘텐츠를 저장할 딕셔너리 초기화
    combined_recommendations = {}

    # 모든 추천 콘텐츠 ID 집합 생성 & 중복 제거
    all_recommendations = set([content_id for content_id, _ in mbti_recommendations] +
                              [content_id for content_id, _ in similar_content_recommendations])

    # 이전에 추천된 콘텐츠와 사용자의 초기 선호 콘텐츠 필터링
    filtered_content_ids = {content_id for content_id in all_recommendations
                            if content_id not in previous_recommendations and
                            content_id not in all_preferred_contents}

    # 필터링된 콘텐츠의 추천 점수 초기화
    for content_id in filtered_content_ids:
        combined_recommendations[content_id] = 0

    # MBTI 유사도 점수 보정
    mbti_scores = [score for content_id, score in mbti_recommendations if content_id in filtered_content_ids]
    mbti_std = np.std(mbti_scores) # std
    mbti_scores_normalized = [score / mbti_std for score in mbti_scores]

    # 필터링된 콘텐츠의 MBTI 점수 가중치 적용
    for (content_id, _), score in zip(mbti_recommendations, mbti_scores_normalized):
        if content_id in filtered_content_ids:
            combined_recommendations[content_id] += weight_mbti * score

    # 선호 콘텐츠 유사도 점수 보정
    similar_scores = [score for content_id, score in similar_content_recommendations if content_id in filtered_content_ids]
    similar_std = np.std(similar_scores)
    similar_scores_normalized = [score / similar_std for score in similar_scores]

    # 필터링된 콘텐츠의 유사 콘텐츠 점수 가중치 적용
    for (content_id, _), score in zip(similar_content_recommendations, similar_scores_normalized):
        if content_id in filtered_content_ids:
            combined_recommendations[content_id] += weight_similar * score

    # 모델 점수 보정
    # 필터링된 콘텐츠 인덱스
    content_indices = [content.index[content['Title'] == content_id][0] for content_id in filtered_content_ids]
    if content_indices:
        # 모델에 입력할 스케일링된 피쳐
        content_features = cbf_model_input_scaled[content_indices]
        model_scores = model.predict(content_features).flatten()
        model_std = np.std(model_scores)
        model_scores_normalized = [score / model_std for score in model_scores]

        # 필터링된 콘텐츠의 모델 점수 가중치 적용
        for content_id, model_score in zip(filtered_content_ids, model_scores_normalized):
            combined_recommendations[content_id] += weight_model * model_score

    # 대중성 점수 보정
    # 필터링된 콘텐츠의 대중성 점수
    popularity_scores = [content.loc[content['Title'] == content_id, 'Normalized Popularity Score'].values[0] for content_id in filtered_content_ids]
    popularity_std = np.std(popularity_scores)
    popularity_scores_normalized = [score / popularity_std for score in popularity_scores]

    # 필터링된 콘텐츠의 대중성 점수 가중치 적용
    for content_id, popularity_score in zip(filtered_content_ids, popularity_scores_normalized):
        combined_recommendations[content_id] += weight_popularity * popularity_score

    # 평점이 3.5 이상인 콘텐츠만 필터링
    filtered_recommendations = {content_id: score for content_id, score in combined_recommendations.items()
                                if content.loc[content['Title'] == content_id, 'Rating Value'].values[0] >= 3.5}

    # 점수 내림차순으로 정렬 -> 최종 추천 콘텐츠 ID 리턴
    final_recommendations = sorted(filtered_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [content_id for content_id, _ in final_recommendations[:top_n]]

In [ ]:
# 사용자 입력 추천 함수
def user_input_recommendation(mbti_type, preferred_contents, content, cbf_model_input_scaled):
    user_embedding = mbti_embeddings_dict[mbti_type]

    # 최적화된 결합 추천 시스템 실행
    recommendations = recommend_contents_combined(user_embedding, preferred_contents, contents_embeddings_dict, best_gbm, content, cbf_model_input_scaled)

    return recommendations

In [ ]:
# 예시 사용자 입력 데이터
new_user_mbti = "INTP"
new_user_preferred_contents = ['어벤져스: 엔드게임', '어메이징 스파이더맨']

# 최적화된 결합 추천 시스템 실행
start_time = time.time()
optimized_recommendations = user_input_recommendation(new_user_mbti, new_user_preferred_contents, content, cbf_model_input_scaled)
end_time = time.time()
print("최적화된 결합 추천 콘텐츠 목록:", optimized_recommendations)
print(f"최적화된 추천 시스템 실행 시간: {end_time - start_time:.2f} 초")

최적화된 결합 추천 콘텐츠 목록: ['매트릭스', '스파이더맨: 파 프롬 홈', '캡틴 아메리카: 윈터 솔져', '아이언맨 3', '스파이더맨 2', '스파이더맨', '나비 효과', '종말의 세라프', '프린세스 스타의 모험일기', '가디언즈 오브 갤럭시 Vol. 2', '시도니아의 기사 극장판', '노매드랜드', '중계지극해청뢰', '맨 인 블랙', '콘택트', '레미제라블: 뮤지컬 콘서트', '마법소녀를 동경해서', '바스터즈: 거친 녀석들', '부당거래', '나키의 저주: 용의 부활']
최적화된 추천 시스템 실행 시간: 1.08 초


In [ ]:
# 첫 번째 추천으로부터 재추천
liked_contents_input = input("첫 번째 추천 목록에서 선호하는 콘텐츠를 쉼표로 구분하여 입력하세요: ")
liked_contents = [content.strip() for content in liked_contents_input.split(',')]

previous_recommendations = optimized_recommendations
new_preferred_contents = liked_contents  # 사용자 입력을 기반으로 새로운 선호 콘텐츠 목록 생성

# 재추천 시스템 실행 시간 측정
start_time = time.time()
new_user_embedding = mbti_embeddings_dict[new_user_mbti]
second_recommendations = recommend_contents_combined_excluding(new_user_embedding, new_preferred_contents, previous_recommendations, new_user_preferred_contents, contents_embeddings_dict, best_gbm, content, cbf_model_input_scaled)
end_time = time.time()

print("재추천 콘텐츠 목록:", second_recommendations)
print(f"재추천 시스템 실행 시간: {end_time - start_time:.2f} 초")

첫 번째 추천 목록에서 선호하는 콘텐츠를 쉼표로 구분하여 입력하세요: 매트릭스, 스파이더맨: 파 프롬 홈
재추천 콘텐츠 목록: ['어벤져스', '스파이더맨 3', '오블리비언', '토르: 다크 월드', '어메이징 스파이더맨 2', '피니와 퍼브 무비: 2차원을 넘어서', '메이즈 러너', '매트릭스 3: 레볼루션', '맨 인 블랙 3', '어벤져스: 에이지 오브 울트론', '미스 페레그린과 이상한 아이들의 집', '터닝메카드', '더 기버: 기억전달자', '트랜스포머 프라임 비스트헌터: 프레데콘 라이징', '머나먼 세상속으로', '나소흑전기: 첫만남편', '장화신은 고양이: 끝내주는 모험', '드래곤볼 슈퍼', '낙원추방', '늑대소년']
재추천 시스템 실행 시간: 1.31 초


추천 개선을 위한 각 MBTI 유형의 TOP_20 추천 리스트 확인

In [ ]:
# 16가지 MBTI 유형
mbti_types = ["INTP", "ENTP", "INTJ", "ENTJ", "INFJ", "ENFJ", "INFP", "ENFP",
              "ISTJ", "ESTJ", "ISFJ", "ESFJ", "ISTP", "ESTP", "ISFP", "ESFP"]

# 각 MBTI 유형에 대한 추천 리스트를 저장할 딕셔너리
mbti_recommendations = {}

# 각 MBTI 유형에 대해 추천 리스트 생성
for mbti_type in mbti_types:
    user_embedding = mbti_embeddings_dict[mbti_type]
    recommendations = recommend_contents_by_mbti(user_embedding, contents_embeddings_dict, top_n=20)
    mbti_recommendations[mbti_type] = recommendations

# 각 MBTI 유형의 추천 리스트 출력
for mbti_type, recommendations in mbti_recommendations.items():
    print(f"MBTI 유형: {mbti_type}")
    for rank, (title, score) in enumerate(recommendations, start=1):
        print(f"{rank}. {title} (Score: {score:.4f})")
    print("\n")

MBTI 유형: INTP
1. 맨 인 블랙 (Score: 0.5141)
2. 시리얼 킬러 연쇄살인범 (Score: 0.5112)
3. 사경 (Score: 0.5039)
4. 뮤지엄 (Score: 0.5012)
5. 마법소녀를 동경해서 (Score: 0.5012)
6. 외계+인 2부 (Score: 0.4962)
7. 천재도둑 미스터A (Score: 0.4917)
8. 엄마와 나 그리고 나의 커밍아웃 (Score: 0.4888)
9. 더 플레인 노랜딩 (Score: 0.4868)
10. 빙하: 살인의 추억 (Score: 0.4851)
11. 칠검 2 - 백발마녀전 (Score: 0.4818)
12. VIP (Score: 0.4815)
13. 비스트 스토커 (Score: 0.4810)
14. 노 맨 오브 갓 (Score: 0.4800)
15. 킬러 인 하이스쿨 (Score: 0.4800)
16. 블랙 앤 화이트: 던 오브 저스티스 (Score: 0.4784)
17. 나키의 저주: 용의 부활 (Score: 0.4777)
18. 폭렬성시 (Score: 0.4761)
19. 전쟁 (Score: 0.4758)
20. 바스터즈: 거친 녀석들 (Score: 0.4747)


MBTI 유형: ENTP
1. 엄마와 나 그리고 나의 커밍아웃 (Score: 0.5439)
2. 시리얼 킬러 연쇄살인범 (Score: 0.5242)
3. 어떻게든 되는 나날 (Score: 0.5171)
4. 너를 줍다 (Score: 0.5107)
5. 아적자위여해 : 나의 고슴도치 그녀 (Score: 0.5074)
6. 오버 더 레인보우 (Score: 0.5067)
7. 다운레인지 (Score: 0.5024)
8. 고이 잠드소서 (Score: 0.4995)
9. 당신이 잠든 사이 (Score: 0.4982)
10. 령 (Score: 0.4928)
11. 요출장안 (Score: 0.4895)
12. 파 프롬 헤븐 (Score: 0.4888)
13. 회사원 (Score: 0.4883)
14. 사경 (Score

대중성 점수 필터링 처리 이후의 각 MBTI 유형의 TOP_20 추천 리스트 확인

In [ ]:
def recommend_contents_combined_with_popularity_filter(user_embedding, preferred_contents, content_embeddings_dict, model, content, cbf_model_input_scaled, top_n=20):
    weight_mbti = 0.4
    weight_similar = 0.5
    weight_model = 0.1

    # 대중성 점수 필터링 (Normalized Popularity Score 기준 상위 90%)
    popularity_threshold = content['Normalized Popularity Score'].quantile(0.9)
    popular_content = content[content['Normalized Popularity Score'] >= popularity_threshold]
    popular_content_embeddings_dict = {title: emb for title, emb in content_embeddings_dict.items() if title in popular_content['Title'].values}

    mbti_recommendations = recommend_contents_by_mbti(user_embedding, popular_content_embeddings_dict, top_n=100)
    similar_content_recommendations = recommend_similar_contents(preferred_contents, popular_content_embeddings_dict, top_n=100)

    combined_recommendations = {}

    # MBTI 유사도 점수 보정
    mbti_scores = [score for _, score in mbti_recommendations]
    if len(mbti_scores) > 1:
        mbti_std = np.std(mbti_scores)
        mbti_scores_normalized = [score / mbti_std for score in mbti_scores] if mbti_std != 0 else mbti_scores
    else:
        mbti_scores_normalized = mbti_scores

    for (content_id, _), score in zip(mbti_recommendations, mbti_scores_normalized):
        if content_id not in combined_recommendations:
            combined_recommendations[content_id] = 0
        combined_recommendations[content_id] += weight_mbti * score

    # 선호 콘텐츠 유사도 점수 보정
    similar_scores = [score for _, score in similar_content_recommendations]
    if len(similar_scores) > 1:
        similar_std = np.std(similar_scores)
        similar_scores_normalized = [score / similar_std for score in similar_scores] if similar_std != 0 else similar_scores
    else:
        similar_scores_normalized = similar_scores

    for (content_id, _), score in zip(similar_content_recommendations, similar_scores_normalized):
        if content_id not in combined_recommendations:
            combined_recommendations[content_id] = 0
        combined_recommendations[content_id] += weight_similar * score

    # 모델 점수 보정
    content_indices = [content.index[content['Title'] == content_id][0] for content_id in combined_recommendations.keys() if content_id in content['Title'].values]
    if content_indices:
        content_features = cbf_model_input_scaled[content_indices]
        model_scores = model.predict(content_features).flatten()
        if len(model_scores) > 1:
            model_std = np.std(model_scores)
            model_scores_normalized = [score / model_std for score in model_scores] if model_std != 0 else model_scores
        else:
            model_scores_normalized = model_scores

        for content_id, model_score in zip(combined_recommendations.keys(), model_scores_normalized):
            combined_recommendations[content_id] += weight_model * model_score

    filtered_recommendations = {content_id: score for content_id, score in combined_recommendations.items()
                                if content.loc[content['Title'] == content_id, 'Rating Value'].values[0] >= 3.5}

    final_recommendations = sorted(filtered_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [content_id for content_id, _ in final_recommendations[:top_n]]

# 16가지 MBTI 유형
mbti_types = ["INTP", "ENTP", "INTJ", "ENTJ", "INFJ", "ENFJ", "INFP", "ENFP",
              "ISTJ", "ESTJ", "ISFJ", "ESFJ", "ISTP", "ESTP", "ISFP", "ESFP"]

# 각 MBTI 유형에 대한 추천 리스트를 저장할 딕셔너리
mbti_recommendations = {}

# 각 MBTI 유형에 대해 추천 리스트 생성
for mbti_type in mbti_types:
    user_embedding = mbti_embeddings_dict[mbti_type]
    recommendations = recommend_contents_combined_with_popularity_filter(user_embedding, [], contents_embeddings_dict, best_gbm, content, cbf_model_input_scaled, top_n=20)
    mbti_recommendations[mbti_type] = recommendations

# 각 MBTI 유형의 추천 리스트 출력
for mbti_type, recommendations in mbti_recommendations.items():
    print(f"MBTI 유형: {mbti_type}")
    for rank, title in enumerate(recommendations, start=1):
        print(f"{rank}. {title}")
    print("\n")

MBTI 유형: INTP
1. 맨 인 블랙
2. 바스터즈: 거친 녀석들
3. 아메리칸 사이코
4. 부당거래
5. 첫 키스만 50번째
6. 걸캅스
7. 감시자들
8. 늑대소년
9. 메멘토
10. 수상한 그녀
11. 그 남자의 기억법
12. 몬스터 주식회사
13. 나의 히어로 아카데미아
14. 브이 포 벤데타
15. 신세계
16. 말아톤
17. 원더
18. 이미테이션 게임
19. 달의 연인 - 보보경심 려
20. 학교 2017


MBTI 유형: ENTP
1. 메멘토
2. 헤어질 결심
3. 파이트 클럽
4. 늑대소녀와 흑왕자
5. 시애틀의 잠 못 이루는 밤
6. 그 남자의 기억법
7. 연애의 온도
8. 주술회전
9. 하나와 앨리스
10. 김종욱 찾기
11. 내 아내의 모든 것
12. 그 시절, 우리가 좋아했던 소녀
13. 소스 코드
14. 사랑의 블랙홀
15. 데자뷰
16. 옥탑방 고양이
17. 시간을 달리는 소녀
18. 원더풀 라이프
19. 잭 리처
20. 부당거래


MBTI 유형: INTJ
1. 맨 인 블랙
2. 신과함께-죄와 벌
3. 더 기버: 기억전달자
4. 마이너리티 리포트
5. 늑대소년
6. 공동경비구역 JSA
7. 몬스터 주식회사
8. 주술회전
9. 인셉션
10. 본 얼티메이텀
11. 바스터즈: 거친 녀석들
12. 엣지 오브 투모로우
13. 짱구는 못말려 극장판: 전설을 부르는 춤을 춰라, 아미고!
14. 스파이더맨: 뉴 유니버스
15. 청년경찰
16. 검사외전
17. 소스 코드
18. 엑스맨
19. 메멘토
20. 거인


MBTI 유형: ENTJ
1. 아메리칸 사이코
2. 당신, 거기 있어줄래요
3. 애드 아스트라
4. 아이, 로봇
5. 아는 형님
6. 극비수사
7. 메멘토
8. 인크레더블
9. 콘택트
10. 아멜리에
11. 프리즈너스
12. 어메이징 스파이더맨
13. 검사외전
14. 하나와 앨리스
15. 천사와 악마
16. 인셉션
17. 스파이더맨
18. 사카모토입니다만?
19. 해리 포터와 죽음의 성물 2
20. 21


MBTI 유형: INF

- 대중성 점수 가중치를 두는 것 보다 대중성 점수로 필터링을 한 이후에 각각의 유사도 점수를 구하는 것이 더 정확하고 적합한 추천을 가능하게 할 것이라고 사료됨

**최종 추천 시스템**

In [ ]:
# 최종 추천 시스템
def recommend_contents_combined_with_popularity_filter(user_embedding, preferred_contents, content_embeddings_dict, model, content, cbf_model_input_scaled, top_n=20):
    weight_mbti = 0.4
    weight_similar = 0.5
    weight_model = 0.1

    # 대중성 점수 필터링 (Normalized Popularity Score 기준 상위 90%)
    popularity_threshold = content['Normalized Popularity Score'].quantile(0.9)
    popular_content = content[content['Normalized Popularity Score'] >= popularity_threshold]

    # 사용자가 선호하는 콘텐츠는 필터링에서 제외하고 포함시킴
    popular_content_embeddings_dict = {title: emb for title, emb in content_embeddings_dict.items() if title in popular_content['Title'].values or title in preferred_contents}

    mbti_recommendations = recommend_contents_by_mbti(user_embedding, popular_content_embeddings_dict, top_n=100)

    # 유사도 계산
    similar_contents = {}
    for preferred_content in preferred_contents:
        if preferred_content in content_embeddings_dict:
            preferred_embedding = content_embeddings_dict[preferred_content]
            for other_content, other_embedding in popular_content_embeddings_dict.items():
                if other_content not in preferred_contents:
                    similarity = np.dot(preferred_embedding, other_embedding) / (np.linalg.norm(preferred_embedding) * np.linalg.norm(other_embedding))
                    if other_content not in similar_contents:
                        similar_contents[other_content] = 0
                    similar_contents[other_content] += similarity

    similar_content_recommendations = sorted(similar_contents.items(), key=lambda x: x[1], reverse=True)[:100]

    combined_recommendations = {}

    # MBTI 유사도 점수 보정
    mbti_scores = [score for _, score in mbti_recommendations]
    mbti_std = np.std(mbti_scores)
    mbti_scores_normalized = [score / mbti_std for score in mbti_scores]

    for (content_id, _), score in zip(mbti_recommendations, mbti_scores_normalized):
        if content_id not in combined_recommendations:
            combined_recommendations[content_id] = 0
        combined_recommendations[content_id] += weight_mbti * score

    # 선호 콘텐츠 유사도 점수 보정
    similar_scores = [score for _, score in similar_content_recommendations]
    similar_std = np.std(similar_scores)
    similar_scores_normalized = [score / similar_std for score in similar_scores]

    for (content_id, _), score in zip(similar_content_recommendations, similar_scores_normalized):
        if content_id not in combined_recommendations:
            combined_recommendations[content_id] = 0
        combined_recommendations[content_id] += weight_similar * score

    # 모델 점수 보정
    content_indices = [content.index[content['Title'] == content_id][0] for content_id in combined_recommendations.keys()]
    if content_indices:
        content_features = cbf_model_input_scaled[content_indices]
        model_scores = model.predict(content_features).flatten()
        model_std = np.std(model_scores)
        model_scores_normalized = [score / model_std for score in model_scores]

        for content_id, model_score in zip(combined_recommendations.keys(), model_scores_normalized):
            combined_recommendations[content_id] += weight_model * model_score

    filtered_recommendations = {content_id: score for content_id, score in combined_recommendations.items()
                                if content.loc[content['Title'] == content_id, 'Rating Value'].values[0] >= 3.5}

    final_recommendations = sorted(filtered_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [content_id for content_id, _ in final_recommendations[:top_n]]

# 예시 사용자 입력 데이터
new_user_mbti = "INTP"
new_user_preferred_contents = ['해리 포터와 혼혈왕자', '신과함께-인과 연', '미녀는 괴로워', '마이데몬']

# 사용자 임베딩
user_embedding = mbti_embeddings_dict[new_user_mbti]

# 최적화된 결합 추천 시스템 실행
start_time = time.time()
filtered_recommendations = recommend_contents_combined_with_popularity_filter(user_embedding, new_user_preferred_contents, contents_embeddings_dict, best_gbm, content, cbf_model_input_scaled, top_n=20)
end_time = time.time()

print("대중성 필터링을 추가한 추천 콘텐츠 목록:", filtered_recommendations)
print(f"추천 시스템 실행 시간: {end_time - start_time:.2f} 초")

대중성 필터링을 추가한 추천 콘텐츠 목록: ['달의 연인 - 보보경심 려', '본 얼티메이텀', '소스 코드', '해리 포터와 죽음의 성물 2', '반지의 제왕: 반지 원정대', '맨 인 블랙', '콘스탄틴', '헝거게임: 캣칭 파이어', '서유기: 선리기연', '드래곤 길들이기 3', '블랙 위도우', '반지의 제왕: 두 개의 탑', '어바웃 타임', '반지의 제왕: 왕의 귀환', '말레피센트', '브루스 올마이티', '스파이더맨 2', '겨울왕국', '원펀맨', '하이힐']
추천 시스템 실행 시간: 1.36 초


**재추천을 포함한 최종 추천 시스템**

In [ ]:
# 최종 추천 시스템 (재추천 포함)
def recommend_contents_combined_excluding_with_popularity_filter(user_embedding, preferred_contents, previous_recommendations, initial_preferred_contents, content_embeddings_dict, model, content, cbf_model_input_scaled, top_n=20):
    # 가중치 설정
    weight_mbti = 0.4
    weight_similar = 0.5
    weight_model = 0.1

    # 초기 선호 콘텐츠와 현재 선호 콘텐츠 합치기
    all_preferred_contents = list(set(preferred_contents + initial_preferred_contents))

    # 대중성 점수 필터링 (Normalized Popularity Score 기준 상위 90%)
    popularity_threshold = content['Normalized Popularity Score'].quantile(0.9)
    popular_content = content[content['Normalized Popularity Score'] >= popularity_threshold]

    # 사용자가 선호하는 콘텐츠는 필터링에서 제외하고 포함시킴
    popular_content_embeddings_dict = {title: emb for title, emb in content_embeddings_dict.items() if title in popular_content['Title'].values or title in all_preferred_contents}

    # MBTI와 유사 콘텐츠 추천 상위 100개 가져오기
    mbti_recommendations = recommend_contents_by_mbti(user_embedding, popular_content_embeddings_dict, top_n=100)

    # 유사도 계산
    similar_contents = {}
    for preferred_content in all_preferred_contents:
        if preferred_content in content_embeddings_dict:
            preferred_embedding = content_embeddings_dict[preferred_content]
            for other_content, other_embedding in popular_content_embeddings_dict.items():
                if other_content not in all_preferred_contents:
                    similarity = np.dot(preferred_embedding, other_embedding) / (np.linalg.norm(preferred_embedding) * np.linalg.norm(other_embedding))
                    if other_content not in similar_contents:
                        similar_contents[other_content] = 0
                    similar_contents[other_content] += similarity

    similar_content_recommendations = sorted(similar_contents.items(), key=lambda x: x[1], reverse=True)[:100]

    # 추천 콘텐츠를 저장할 딕셔너리 초기화
    combined_recommendations = {}

    # 모든 추천 콘텐츠 ID 집합 생성 & 중복 제거
    all_recommendations = set([content_id for content_id, _ in mbti_recommendations] +
                              [content_id for content_id, _ in similar_content_recommendations])

    # 이전에 추천된 콘텐츠와 사용자의 초기 선호 콘텐츠 필터링
    filtered_content_ids = {content_id for content_id in all_recommendations
                            if content_id not in previous_recommendations and
                            content_id not in all_preferred_contents}

    # 필터링된 콘텐츠의 추천 점수 초기화
    for content_id in filtered_content_ids:
        combined_recommendations[content_id] = 0

    # MBTI 유사도 점수 보정
    mbti_scores = [score for content_id, score in mbti_recommendations if content_id in filtered_content_ids]
    mbti_std = np.std(mbti_scores)
    mbti_scores_normalized = [score / mbti_std for score in mbti_scores]

    # 필터링된 콘텐츠의 MBTI 점수 가중치 적용
    for (content_id, _), score in zip(mbti_recommendations, mbti_scores_normalized):
        if content_id in filtered_content_ids:
            combined_recommendations[content_id] += weight_mbti * score

    # 선호 콘텐츠 유사도 점수 보정
    similar_scores = [score for content_id, score in similar_content_recommendations if content_id in filtered_content_ids]
    similar_std = np.std(similar_scores)
    similar_scores_normalized = [score / similar_std for score in similar_scores]

    # 필터링된 콘텐츠의 유사 콘텐츠 점수 가중치 적용
    for (content_id, _), score in zip(similar_content_recommendations, similar_scores_normalized):
        if content_id in filtered_content_ids:
            combined_recommendations[content_id] += weight_similar * score

    # 모델 점수 보정
    # 필터링된 콘텐츠 인덱스
    content_indices = [content.index[content['Title'] == content_id][0] for content_id in filtered_content_ids]
    if content_indices:
        # 모델에 입력할 스케일링된 피쳐
        content_features = cbf_model_input_scaled[content_indices]
        model_scores = model.predict(content_features).flatten()
        model_std = np.std(model_scores)
        model_scores_normalized = [score / model_std for score in model_scores]

        # 필터링된 콘텐츠의 모델 점수 가중치 적용
        for content_id, model_score in zip(filtered_content_ids, model_scores_normalized):
            combined_recommendations[content_id] += weight_model * model_score

    # 평점이 3.5 이상인 콘텐츠만 필터링
    filtered_recommendations = {content_id: score for content_id, score in combined_recommendations.items()
                                if content.loc[content['Title'] == content_id, 'Rating Value'].values[0] >= 3.5}

    # 점수 내림차순으로 정렬 -> 최종 추천 콘텐츠 ID 리턴
    final_recommendations = sorted(filtered_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [content_id for content_id, _ in final_recommendations[:top_n]]

# 예시 사용자 입력 데이터
new_user_mbti = "INFP"
new_user_preferred_contents = ['라라랜드', '500일의 썸머', '인터스텔라', '이터널 선샤인', '헤어질 결심', '시카리오: 암살자들의 도시', '남한산성']

# 첫 번째 추천 시스템 실행
start_time = time.time()
user_embedding = mbti_embeddings_dict[new_user_mbti]
initial_recommendations = recommend_contents_combined_with_popularity_filter(user_embedding, new_user_preferred_contents, contents_embeddings_dict, best_gbm, content, cbf_model_input_scaled, top_n=20)
end_time = time.time()
print("첫 번째 추천 콘텐츠 목록:", initial_recommendations)
print(f"추천 시스템 실행 시간: {end_time - start_time:.2f} 초")

# 첫 번째 추천으로부터 재추천
liked_contents_input = input("첫 번째 추천 목록에서 선호하는 콘텐츠를 쉼표로 구분하여 입력하세요: ")
liked_contents = [content.strip() for content in liked_contents_input.split(',')]

previous_recommendations = initial_recommendations
new_preferred_contents = liked_contents  # 사용자 입력을 기반으로 새로운 선호 콘텐츠 목록 생성

# 재추천 시스템 실행 시간 측정
start_time = time.time()
second_recommendations = recommend_contents_combined_excluding_with_popularity_filter(user_embedding, new_preferred_contents, previous_recommendations, new_user_preferred_contents, contents_embeddings_dict, best_gbm, content, cbf_model_input_scaled, top_n=20)
end_time = time.time()

print("재추천 콘텐츠 목록:", second_recommendations)
print(f"재추천 시스템 실행 시간: {end_time - start_time:.2f} 초")

첫 번째 추천 콘텐츠 목록: ['네 멋대로 해라', '브루스 올마이티', '늑대소년', '주술회전', '지금 만나러 갑니다', '럭키', '데자뷰', '서약', '러브 액츄얼리', '꿈의 제인', '매기스 플랜', '레터스 투 줄리엣', '우리들', '블루 발렌타인', '색, 계', '왕의 남자', '캐리비안의 해적: 망자의 함', '본 슈프리머시', '라이온 킹', '나우 이즈 굿']
추천 시스템 실행 시간: 1.20 초
첫 번째 추천 목록에서 선호하는 콘텐츠를 쉼표로 구분하여 입력하세요: 지금 만나러 갑니다, 러브 액츄얼리, 나우 이즈 굿, 꿈의 제인, 블루 발렌타인
재추천 콘텐츠 목록: ['말아톤', '늑대소녀와 흑왕자', '내 아내의 모든 것', '그녀', '바닐라 스카이', '아저씨', '지금, 만나러 갑니다', '메종 드 히미코', '시월애', '도망자', '클래식', '너의 이름은.', '시애틀의 잠 못 이루는 밤', '인생은 아름다워', '우리도 사랑일까', '죽거나 혹은 나쁘거나', '와일드', '히로인 실격', '존 윅', '검은 사제들']
재추천 시스템 실행 시간: 1.23 초


# 추천 시스템 평가

- NDCG(Normalized Discounted Cumulative Gain)
  - 추천 시스템의 성능을 평가하는데 널리 사용되는 지표
  - 추천 항목의 순위와 관련성 모두 고려하여 측정
  - NDCG@k는 상위 k개의 추천 항목에 대해 NDCG를 계산한 값


In [49]:
# DCG 계산 함수
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(r / np.log2(np.arange(2, r.size + 2)))
    return 0.

# NDCG 계산 함수
def ndcg_at_k(r, k):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

# 사용자별 실제 선호 콘텐츠와 추천 결과
actual_preferences = {
    'user1': ['해리 포터와 죽음의 성물 2', '맨 인 블랙', '콘스탄틴', '헝거게임: 캣칭 파이어', '서유기: 선리기연','어바웃 타임', '스파이더맨 2', '겨울왕국'],
    'user2': ['검사외전', '부당거래', '월드워Z', '그 시절, 우리가 좋아했던 소녀', '늑대소년'],
    'user3': ['월드워Z', '매트릭스 3: 레볼루션', '폼포코 너구리 대작전', '엣지 오브 투모로우', '가디언즈 오브 갤럭시 Vol. 2', '미이라', '토르: 다크 월드'],
    'user4': ['메멘토', '이미테이션 게임','에놀라 홈즈', '겨울왕국 2'],
    'user5': ['주술회전', '너의 이름은.'],
    'user6': ['쩐의 전쟁', '미녀와 야수', '메이즈 러너', '너의 이름은.', '소울', '타이타닉'],
    'user7': ['너의 이름은.', '하이큐!!', '나츠메 우인장', '심야식당 2'],
    'user8': ['맨 인 블랙', '아토믹 블론드', '아이언맨 3', '인셉션', '매트릭스 3: 레볼루션', '스파이더맨: 파 프롬 홈', '맨 인 블랙 3'],
    'user9': ['해리 포터와 비밀의 방', '러브레터', '해리 포터와 아즈카반의 죄수', '어메이징 스파이더맨', '런닝맨', '해리 포터와 불사조 기사단', '스파이더맨 2'],
    'user10': ['500일의 썸머', '짱구는 못말려 극장판: 액션가면 VS 그래그래 마왕', '드래곤 길들이기 2', '미스 페레그린과 이상한 아이들의 집', '소울', '어메이징 스파이더맨 2', '가디언즈 오브 갤럭시 Vol. 2', '콘스탄틴'],
    'user11': ['지금 만나러 갑니다', '러브 액츄얼리', '꿈의 제인', '매기스 플랜', '우리들', '블루 발렌타인', '색, 계', '캐리비안의 해적: 망자의 함', '본 슈프리머시', '나우 이즈 굿']
}

recommendations = {
    'user1': ['달의 연인 - 보보경심 려', '본 얼티메이텀', '소스 코드', '해리 포터와 죽음의 성물 2', '반지의 제왕: 반지 원정대', '맨 인 블랙', '콘스탄틴', '헝거게임: 캣칭 파이어', '서유기: 선리기연', '드래곤 길들이기 3', '블랙 위도우', '반지의 제왕: 두 개의 탑', '어바웃 타임', '반지의 제왕: 왕의 귀환', '말레피센트', '브루스 올마이티', '스파이더맨 2', '겨울왕국', '원펀맨', '하이힐'],
    'user2': ['검사외전', '내가 살인범이다', '네 멋대로 해라', '주술회전', '레터스 투 줄리엣', '13층', '헤어질 결심', '브루스 올마이티', '콘택트', '부당거래', '시애틀의 잠 못 이루는 밤', '판의 미로: 오필리아와 세개의 열쇠', '무간도 2: 혼돈의 시대', '짱구는 못말려 극장판: 태풍을 부르는 장엄한 전설의 전투', '월드워Z', '사랑의 블랙홀', '죄 많은 소녀', '그 시절, 우리가 좋아했던 소녀', '늑대소년', '옥탑방 고양이'],
    'user3': ['월드워Z', '판의 미로: 오필리아와 세개의 열쇠', '매트릭스 3: 레볼루션', '주술회전', '라이온 킹', '폼포코 너구리 대작전', '미녀와 야수', '엣지 오브 투모로우', '가디언즈 오브 갤럭시 Vol. 2', '인어공주', '미이라', '토르: 다크 월드', '겨울왕국 2', '서유기: 선리기연', '날씨의 아이', '브이 포 벤데타', '반지의 제왕: 왕의 귀환', '늑대소년', '스타워즈 에피소드 1: 보이지 않는 위험', '짱구는 못말려 극장판: 암흑 마왕 대추적'],
    'user4': ['아토믹 블론드', '바스터즈: 거친 녀석들', '메멘토', '이미테이션 게임', '미션 임파서블', '캡틴 아메리카: 윈터 솔져', '블러드 다이아몬드', '페이스 오프', '에놀라 홈즈', '천사와 악마', '겨울왕국 2', '미션 임파서블: 폴아웃', '스파이더맨 2', '앤트맨', '트루 라이즈', '스파이 브릿지', '혹성탈출: 진화의 시작', '프레스티지', '스타워즈 에피소드 2: 클론의 습격', '앤트맨과 와스프'],
    'user5': ['늑대소년', '주술회전', '데자뷰', '용의자 X의 헌신', '브루스 올마이티', '죄 많은 소녀', '몬스터 주식회사', '플레이스 비욘드 더 파인즈', '너의 이름은.', '서치', '메이즈 러너', '괴물의 아이', '기생수', '나츠메 우인장', '네 멋대로 해라', '서유기: 선리기연', '이 멋진 세계에 축복을!', '토르: 다크 월드', '에이리언: 커버넌트', '콰이어트 플레이스 2'],
    'user6': ['파이트 클럽', '브루스 올마이티', '늑대소년', '주술회전', '팀 버튼의 크리스마스 악몽', '네 멋대로 해라', '쩐의 전쟁', '블루 발렌타인', '인크레더블', '미녀와 야수', '메이즈 러너', '트랜스포머', '보글보글 스폰지밥', '너의 이름은.', '소울', '타이타닉', '콘스탄틴', '기생수', '시스터 액트', '다크 나이트'],
    'user7': ['늑대소년', '주술회전', '짱구는 못말려 극장판: 액션가면 VS 그래그래 마왕', '너의 이름은.', '날씨의 아이', '갓파쿠와 여름방학을', '언어의 정원', '별을 쫓는 아이: 아가르타의 전설', '하이큐!!', '메종 드 히미코', '나츠메 우인장', '짱구는 못말려 극장판: 부리부리 왕국의 보물', '쉘 위 댄스', '세상의 중심에서 사랑을 외치다', '혐오스런 마츠코의 일생', '모모와 다락방의 수상한 요괴들', '심야식당 2', '시', '초속 5센티미터', '말아톤'],
    'user8': ['맨 인 블랙', '몬스터 주식회사', '아토믹 블론드', '본 얼티메이텀', '브이 포 벤데타', '파이트 클럽', '아이언맨 3', '인셉션', '매트릭스 3: 레볼루션', '메이즈 러너', '트랜스포머', '터미네이터 2: 심판의 날', '스파이더맨: 파 프롬 홈', '미스 페레그린과 이상한 아이들의 집', '스파이더맨: 뉴 유니버스', '미션 임파서블: 데드 레코닝 PART ONE', '맨 인 블랙 3', '스파이더맨 3', '본 슈프리머시', '셜록 홈즈: 그림자 게임'],
    'user9': ['말아톤', '늑대소년', '해리 포터와 비밀의 방', '탐정: 더 비기닝', '죄 많은 소녀', '시', '새벽의 연화', '러브레터', '호리미야', '아가씨', '릴리 슈슈의 모든 것', '미녀와 야수', '날씨의 아이', '히로인 실격', '해리 포터와 아즈카반의 죄수', '어메이징 스파이더맨', '이지 A', '런닝맨', '해리 포터와 불사조 기사단', '스파이더맨 2'],
    'user10': ['늑대소년', '주술회전', '팀 버튼의 크리스마스 악몽', '사랑에 대한 모든 것', '500일의 썸머', '몬스터 주식회사', '괴물의 아이', '짱구는 못말려 극장판: 액션가면 VS 그래그래 마왕', '드래곤 길들이기 2', '미스 페레그린과 이상한 아이들의 집', '소울', '토이 스토리', '체인소 맨', '토이 스토리 4', '어메이징 스파이더맨 2', '토르: 다크 월드', '피터 팬', '가디언즈 오브 갤럭시 Vol. 2', '서유기: 선리기연', '콘스탄틴'],
    'user11': ['네 멋대로 해라', '브루스 올마이티', '늑대소년', '주술회전', '지금 만나러 갑니다', '럭키', '데자뷰', '서약', '러브 액츄얼리', '꿈의 제인', '매기스 플랜', '레터스 투 줄리엣', '우리들', '블루 발렌타인', '색, 계', '왕의 남자', '캐리비안의 해적: 망자의 함', '본 슈프리머시', '라이온 킹', '나우 이즈 굿']
}

# 사용자별 NDCG 계산
k = 20
ndcg_scores = []

for user, actual in actual_preferences.items():
    recommended = recommendations[user]
    relevance = [1 if item in actual else 0 for item in recommended]
    ndcg = ndcg_at_k(relevance, k)
    ndcg_scores.append(ndcg)
    print(f"NDCG@{k} for {user}: {ndcg}")

# 평균 NDCG
average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG@{k}: {average_ndcg:.4f}")

NDCG@20 for user1: 0.6259276783324935
NDCG@20 for user2: 0.6803051600751515
NDCG@20 for user3: 0.8306441614162442
NDCG@20 for user4: 0.5897274270918277
NDCG@20 for user5: 0.5714285040141098
NDCG@20 for user6: 0.49988395779549066
NDCG@20 for user7: 0.48815540478337316
NDCG@20 for user8: 0.811514705905686
NDCG@20 for user9: 0.5510096187094674
NDCG@20 for user10: 0.5778310654895317
NDCG@20 for user11: 0.6002760303981841
Average NDCG@20: 0.6206


- 평균 NDCG@20 점수가 0.6206이라는 것은 추천 시스템이 평균적으로 상위 20개의 추천 항목에서 62.06%의 이상적인 관련성을 가진다는 것을 의미합니다.
- 이는 추천 시스템이 꽤 효과적으로 작동하고 있음을 시사합니다.
- 하지만 아직 낮은 점수를 기록한 사용자들에 대한 개선이 가능하고 필요합니다.

# 7. MVTI 추천 시스템의 특징 및 장점

**특징 및 장점**
- **다중 기준 결합(하이브리드 추천)**: MBTI 임베딩 유사도, 선호 콘텐츠 임베딩 유사도, 모델 예측 점수, 대중성 점수를 결합하여 다양한 측면에서 최적의 콘텐츠를 추천합니다.
- **개인화된 추천**: 사용자의 MBTI 특성과 사용자 선호 콘텐츠를 반영하여 개인화된 추천을 제공합니다.
- **인기 콘텐츠 추천**: 정규화된 대중성 점수를 사용하여 상위 90% 대중성을 가진 콘텐츠를 우선적으로 추천하여 사용자 만족도를 높입니다.
- **동적 추천 시스템(재추천)**: 사용자가 이전 추천 목록에서 선택한 콘텐츠를 반영하고 이전에 추천 받은 콘텐츠는 제외하여 새로운 추천 리스트를 제공합니다.
- **확장 가능성**: 유저 행동 데이터 및 SNS 데이터 등의 다양한 데이터 소스를 통합하여 더 정교한 추천 시스템으로 확장할 수 있습니다.

**평가 결과: Average NDCG@20: 0.6206**
- **평가 지표**: NDCG는 추천된 콘텐츠의 순서와 실제 선호도를 비교하여 점수를 매기는 지표로, 높은 점수일수록 더 정확한 추천을 의미합니다.
- **평균 NDCG@20 점수**: 0.6206은 추천 시스템이 사용자 선호도를 잘 반영하고 있음을 나타냅니다. 이는 추천된 상위 20개의 콘텐츠 중 다수가 실제로 사용자가 선호하는 콘텐츠임을 의미합니다.
- **개선 여지**: 비록 0.6206이라는 점수가 나쁘지 않지만, 더 높은 점수를 목표로 추천 알고리즘의 튜닝 및 추가 기능 개선이 필요할 수 있습니다.

**결론**
: MBTI 추천 시스템은 다양한 추천 기법을 결합하여 사용자 맞춤형 콘텐츠를 정밀하고 다양하게 추천합니다. 이를 위해 MBTI 임베딩 유사도, 선호 콘텐츠 임베딩 유사도, 모델 예측 점수를 결합하여 최적의 콘텐츠를 제공합니다. 또한, 정규화된 대중성 점수를 통해 상위 90%의 대중성을 가진 콘텐츠를 우선적으로 추천하여 사용자 만족도를 높입니다.

이 시스템은 사용자의 MBTI 특성과 선호 콘텐츠를 반영한 개인화된 추천과 사용자의 이전 선택을 반영한 동적 추천을 제공합니다. 이러한 기능은 사용자 경험을 더욱 향상시키며, 유저 행동 데이터와 SNS 데이터를 통합하여 더 정교한 추천 시스템으로 확장할 수 있는 가능성도 갖추고 있습니다.

평가 결과, 평균 NDCG@20 점수는 0.6206으로, 이는 시스템이 사용자 선호도를 잘 반영하고 있음을 나타냅니다. 향후 더 높은 점수를 목표로 알고리즘의 튜닝 및 추가 기능 개선이 필요할 수 있습니다.

종합적으로, MBTI 추천 시스템은 고성능 임베딩 모델과 다양한 데이터 소스를 활용하여 사용자에게 가장 적합한 콘텐츠를 제공하는 효과적인 추천 시스템입니다.







